In [46]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

script_dir = os.path.abspath("")
mymodule_dir = os.path.join(script_dir, "..", "scripts")
sys.path.append(mymodule_dir)
import cleaning
import nlp

pd.set_option("mode.copy_on_write", True)
sns.set_theme()
sns.set_style("white")
%config InlineBackend.figure_format = "retina"

jobs = cleaning.prep(pd.read_pickle("../data/jobs.pkl"), filtered=False, categories_reduced=False)
jobs.head()

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
0,(Junior) Data Scientist (m/w/d),KMH GmbH,Köln,4032,5726,57650956,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Teilzeit, Home Office möglich",Wir suchen in Voll- oder Teilzeit\nWir suchen ...,...,Wir freuen uns über deine Bewerbung!\n\n KMH G...,True,True,True,True,Data Scientist,True,48384,68712,58548.0
1,Junior Data Scientist (m/w/d),TUI Cruises GmbH,Hamburg,3548,5081,57605321,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises ist einer der führenden Kreuzfahrt...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
2,Junior Data Scientist (m/f/x),TUI Cruises GmbH,Hamburg,3548,5081,57605323,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises is one of the leading cruise provi...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
3,(Junior) Data Scientist / Data Analyst (w/m/d),1&1,"Karlsruhe, Montabaur, Maintal",3871,6129,57608864,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,...,,True,False,False,True,Data Scientist,True,46452,73548,60000.0
4,Junior Data Scientist (m/w/d) – Risikomodelle,Sparkassen Rating und Risikosysteme GmbH,Berlin,3710,5161,57181289,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Die Sparkassen Rating und Risikosysteme (SR) i...,...,Herr Emanuel Reitzenstein Tel. +49 30 20672-0 ...,True,False,False,True,Data Scientist,True,44520,61932,53226.0


In [3]:
all_other_jobs = jobs.loc[jobs["Job Kategorie"] == "Andere", :]

In [4]:
other_data_jobs = cleaning.filter(jobs).loc[lambda df: df["Job Kategorie"] == "Andere", :]

In [5]:
merged = all_other_jobs.merge(other_data_jobs, on="JobID", indicator=True, how="left")
non_data_jobs = all_other_jobs.loc[all_other_jobs["JobID"].isin(merged.loc[merged["_merge"] == "left_only", "JobID"])]
non_data_jobs.shape

(5045, 23)

In [60]:
irrelevant_keywords = "senior|software|projektleiter|developerdeveloper|administrator|projektentwickler|accountant|hr|sap|disponent|sous|recruit|controll|courier|vertrieb|driv|personalreferent|einkäufer|softwareentwickl"
interesting_jobs = non_data_jobs[~non_data_jobs["Titel"].str.contains(irrelevant_keywords, flags=re.IGNORECASE, regex=True)]

In [61]:
lemmas = [nlp.tokenize_doc(title) for title in interesting_jobs["Titel"]]

In [62]:
from collections import Counter

all_title_words = [w for doc in lemmas for w in doc]
bow = Counter(all_title_words)
bow.most_common(10)

[('junior', 1453),
 ('special', 212),
 ('developer', 126),
 ('business', 92),
 ('referent', 86),
 ('bereich', 84),
 ('experte', 77),
 ('ingenieur', 76),
 ('mitarbeiter', 76),
 ('gender', 69)]

In [63]:
all_titles = [" ".join(doc) for doc in lemmas]

In [64]:
df_all_titles = pd.DataFrame({"Titel": all_titles})
df_all_titles.loc[~df_all_titles["Titel"].str.contains(irrelevant_keywords, regex=True), "Titel"].value_counts()

Titel
junior talent acquisition special                                                    11
junior treasur                                                                        9
junior buyer                                                                          8
full stack developer                                                                  8
ausbildung fachinformatiker systemintegration                                         7
                                                                                     ..
phd ai ml based credibl simulation model power modul hybrid full electric car div     1
sachgebietsleitung finanz steuern tv unbefristet                                      1
crossmedia redakteur brigitte schwerpunkt femal finance                               1
manufacturing technician                                                              1
junior berater pr social media                                                        1
Name: count, Length: 2156,

In [65]:
interesting_jobs[interesting_jobs["Titel"].str.contains("burda", flags=re.IGNORECASE, regex=True)]

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
